In [23]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [24]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [25]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [26]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_midcp(f_today):

    days_to_monday = (0 - f_today.weekday()) % 7
    nearest_monday = f_today + dt.timedelta(days=days_to_monday)
    f_expiry = nearest_monday
    if nse.valid_days(start_date=nearest_monday, end_date=nearest_monday).empty:
        f_expiry = nearest_monday - dt.timedelta(days=3)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [27]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
bnf_1min = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
# bnf_1min = pd.read_csv('../data/nifty_min.csv')
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year >= 2023) & (bnf_1min['datetime'].dt.month >= 9) ) & (bnf_1min['datetime'].dt.year <= 2024)]
bnf_1min = bnf_1min[
    ((bnf_1min["datetime"].dt.year == 2023) & (bnf_1min["datetime"].dt.month >= 9))
    | (bnf_1min["datetime"].dt.year >= 2024)
]

In [28]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [29]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [30]:
bnf_1min

,datetime,open,high,low,close,volume
149247,2023-09-01 09:15:00,8894.0000,8896.2500,8867.5500,8883.7000,0.0000
149248,2023-09-01 09:16:00,8882.9000,8900.1500,8882.9000,8898.0500,0.0000
149249,2023-09-01 09:17:00,8899.1000,8902.4500,8894.9500,8896.8500,0.0000
149250,2023-09-01 09:18:00,8897.1000,8904.8000,8897.1000,8901.1000,0.0000
149251,2023-09-01 09:19:00,8900.5500,8901.5000,8895.9000,8896.7500,0.0000
...,...,...,...,...,...,...
202282,2024-03-28 15:25:00,10528.9500,10532.4000,10527.5000,10531.7500,0.0000
202283,2024-03-28 15:26:00,10531.7000,10533.9000,10528.6000,10532.8500,0.0000
202284,2024-03-28 15:27:00,10532.9000,10534.9500,10530.8000,10534.4000,0.0000
202285,2024-03-28 15:28:00,10534.2000,10540.3500,10526.3500,10535.1500,0.0000


In [31]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [35]:
def calculate_weekly_ranges(df):
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)
    # Resample to weekly data to get the high and low for each week
    weekly_ranges = df.resample("W-FRI").agg({"high": "max", "low": "min"})
    return weekly_ranges


async def find_breakouts(df, weekly_ranges):

    results = []
    multiplier_to_range = 0.5

    portfolio_value = 10_00_000
    index_leverage = 10

    index_name = "MIDCPNIFTY"
    index_str_for_opt = "midcpnifty"

    weekly_ranges["datetime"] = pd.to_datetime(weekly_ranges["datetime"])
    # weekly_ranges = weekly_ranges[:-1]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(1, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i - 1]
        current_week = weekly_ranges.iloc[i]
        print("Current Week : ", current_week["datetime"].date())
        if current_week["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_week_high = weekly_ranges.iloc[i - 1]["high"]
        previous_week_low = weekly_ranges.iloc[i - 1]["low"]
        previous_week_close = weekly_ranges.iloc[i - 1]["close"]

        current_week_open = weekly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[
            (df["datetime"] >= current_week["datetime"])
            & (df["datetime"] <= (current_week["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data["open"].iloc[0] + addition_range
        low_level = current_week_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_week_data[current_week_data["high"] >= high_level]
        breakout_low = current_week_data[current_week_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 25) * 25)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_midcp(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_high_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            low_breach = current_week_data_after_entry[
                current_week_data_after_entry["low"] < previous_week_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Friday Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 25) * 25)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_midcp(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 30),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_low_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            high_breach = current_week_data_after_entry[
                current_week_data_after_entry["high"] > previous_week_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    if len(ce_df) > 0:
                        # print(ce_df)
                        filtered_df = ce_df.loc[
                            (ce_df["datetime"].dt.date >= exit_time.date())
                            & (ce_df["datetime"].dt.hour >= exit_time.hour)
                            & (ce_df["datetime"].dt.minute >= exit_time.minute),
                            "c"]
                        exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')
                    else:
                        exit_price = float("nan")
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results


async def trade():
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    weekly_ranges = resample(pl.DataFrame(df), "7d", pd.Timedelta(days=5))
    weekly_ranges = weekly_ranges.to_pandas()

    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas


tradebook = await trade()
# tradebook

Current Week :  2023-09-12
Current Week :  2023-09-19
Current Week :  2023-09-26
Current Week :  2023-10-03
Current Week :  2023-10-10
Current Week :  2023-10-17
Current Week :  2023-10-24
Current Week :  2023-10-31
Current Week :  2023-11-07
Current Week :  2023-11-14
Current Week :  2023-11-21
Current Week :  2023-11-28
Current Week :  2023-12-05
Current Week :  2023-12-12
Current Week :  2023-12-19
Current Week :  2023-12-26
Current Week :  2024-01-02
Current Week :  2024-01-09
Current Week :  2024-01-16
Current Week :  2024-01-23
Current Week :  2024-01-30
Current Week :  2024-02-06
Current Week :  2024-02-13
Current Week :  2024-02-20
Current Week :  2024-02-27
Current Week :  2024-03-05
Current Week :  2024-03-12
Current Week :  2024-03-19
Current Week :  2024-03-26


In [33]:
tradebook

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
0,2023-09-05,9208.2000,8971.7500,236.4500,9241.3000,50%,9359.5250,9123.0750,low,9350,2023-09-18,6,MIDCPNIFTY239189350CE,CE,2023-09-12 09:40:00,NaN,2023-09-18 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,1336.8984,NaN,NaN,NaN,NaN
1,2023-09-19,9206.3000,8975.5500,230.7500,9186.9500,50%,9302.3250,9071.5750,low,9300,2023-09-29,1,MIDCPNIFTY23SEP9300CE,CE,2023-09-28 13:14:00,NaN,2023-09-29 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,1344.0860,NaN,NaN,NaN,NaN
2,2023-09-26,9217.4500,8997.7500,219.7000,9112.9000,50%,9222.7500,9003.0500,low,9225,2023-10-09,5,MIDCPNIFTY2310099225CE,CE,2023-10-04 11:49:00,NaN,2023-10-09 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,1355.0136,NaN,NaN,NaN,NaN
3,2023-10-03,9130.5500,8932.3500,198.2000,9057.2500,50%,9156.3500,8958.1500,high,8950,2023-10-16,5,MIDCPNIFTY2310168950PE,PE,2023-10-11 09:15:00,NaN,2023-10-16 15:30:00,NaN,Friday Closing,NaN,NaN,NaN,1396.6480,NaN,NaN,NaN,NaN
4,2023-10-10,9222.5000,9051.7000,170.8000,9206.8500,50%,9292.2500,9121.4500,low,9300,2023-10-23,5,MIDCPNIFTY2310239300CE,CE,2023-10-18 11:12:00,NaN,2023-10-23 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,1344.0860,NaN,NaN,NaN,NaN
5,2023-10-17,9213.7500,8816.4500,397.3000,8863.2500,50%,9061.9000,8664.6000,low,9050,2023-10-30,4,MIDCPNIFTY23OCT9050CE,CE,2023-10-26 09:22:00,NaN,2023-10-30 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,1381.2155,NaN,NaN,NaN,NaN
6,2023-10-24,8922.0500,8576.5000,345.5500,8767.2500,50%,8940.0250,8594.4750,high,8600,2023-11-06,0,MIDCPNIFTY2311068600PE,PE,2023-11-06 09:15:00,NaN,2023-11-06 15:30:00,NaN,Friday Closing,NaN,NaN,NaN,1453.4884,NaN,NaN,NaN,NaN
7,2023-10-31,8973.5500,8710.3000,263.2500,8981.8500,50%,9113.4750,8850.2250,high,8850,2023-11-13,4,MIDCPNIFTY2311138850PE,PE,2023-11-09 09:36:00,NaN,2023-11-13 15:30:00,NaN,Friday Closing,NaN,NaN,NaN,1412.4294,NaN,NaN,NaN,NaN
8,2023-11-07,9272.7000,8923.8000,348.9000,9323.5500,50%,9498.0000,9149.1000,high,9150,2023-11-20,0,MIDCPNIFTY2311209150PE,PE,2023-11-20 10:20:00,NaN,2023-11-20 15:30:00,NaN,Friday Closing,NaN,NaN,NaN,1366.1202,NaN,NaN,NaN,NaN
9,2023-11-21,9533.6000,9437.5500,96.0500,9525.3500,50%,9573.3750,9477.3250,high,9475,2023-12-04,5,MIDCPNIFTY2312049475PE,PE,2023-11-29 09:15:00,NaN,2023-12-04 15:30:00,NaN,Friday Closing,NaN,NaN,NaN,1319.2612,NaN,NaN,NaN,NaN


In [34]:
# positive = tradebook[tradebook['Points']<0]
# positive['Points'].sum()
tradebook["ROI% w cs"].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

0.0

In [22]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [28]:
stats_df8 = pd.DataFrame(
    index=range(2023, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2023, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

/tmp/ipykernel_388085/1548016176.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2023,9.5187,13,92.3077,0.8955,-1.2277,-1.2277,7.7532
2024,16.5698,12,100.0000,1.3808,NaN,0.0000,inf
Overall,26.0885,25.0000,96.0000,1.1382,-1.2277,-1.2277,21.2497


50% MOVE VARIATION FROM CURRENT OPEN, ENTRY AT OPP LEVELS, SL AT PRV. WK H/L

In [29]:
# stats_of_trades = tradebook
# stats_of_trades["Cumulative ROI%"] = stats_of_trades.groupby("Option Type")[
#     "ROI%"
# ].cumsum()
# stats_of_trades["Running Max ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Cumulative ROI%"
# ].cummax()
# stats_of_trades["Drawdown"] = (
#     stats_of_trades["Cumulative ROI%"] - stats_of_trades["Running Max ROI%"]
# )
# max_dd_distribution = (
#     stats_of_trades.groupby("Option Type")["Drawdown"].min().reset_index()
# )
# max_dd_distribution.rename(columns={"Drawdown": "Max Drawdown"}, inplace=True)
# roi_distribution = stats_of_trades.groupby("Option Type")["ROI%"].sum().reset_index()
# grouped = pd.merge(roi_distribution, max_dd_distribution, on="Option Type")
# grouped["ROI/DD Ratio"] = grouped["ROI%"] / grouped["Max Drawdown"].abs()
# grouped

In [30]:
# tradebook.to_csv("MIDCP Final TB JJMS RBOS 65pct var4.csv")
# tradebook2 = pd.read_csv('MIDCP Final TB JJMS RBOS 65pct var4.csv')
tradebook2 = tradebook
tradebook2["Expiry"] = pd.to_datetime(tradebook2["Expiry"])
tradebook2["Entry Time"] = pd.to_datetime(tradebook2["Entry Time"])
tradebook2["Exit Time"] = pd.to_datetime(tradebook2["Exit Time"])

In [31]:
def round_to_nearest_25(value):
    return 25 * round(value / 25)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 2
    index_name = "MIDCPNIFTY"
    index_str_for_opt = "midcpnifty"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_25(row["Strike"] * (1 + hedge_pct / 100))
            if row["Option Type"] == "CE"
            else round_to_nearest_25(row["Strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    df["Hedge Contract"] = df.apply(
        lambda row: get_option_contract_name2(
            index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
        ),
        axis=1,
    )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]["Week"])
        hedge_strike = df.iloc[i]["Hedge Strike"]
        hedge_expiry = df.iloc[i]["Expiry"]
        hedge_opt_type = df.iloc[i]["Option Type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["Entry Time"]
        hedge_exit_datetime = df.iloc[i]["Exit Time"]

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tradebook2)
# tb_with_hedge

2023-09-05
3.45 0.05
2023-09-19
1.35 0.05
2023-09-26
2.25 0.05
2023-10-03
4.15 0.05
2023-10-10
1.55 0.1
2023-10-17
1.85 0.05
2023-10-24
0.1 0.05
2023-10-31
0.9 0.05
2023-11-07
0.05 0.05
2023-11-21
3.4 0.05
2023-12-05
1.5 0.05
2023-12-05
3.4 3.0
2023-12-12
2.4 0.05
2023-12-26
1.6 0.1
2023-12-26
5.65 1.8
2024-01-02
2.45 0.1
2024-01-09
3.8 0.2
2024-01-16
2.55 0.05
2024-01-30
6.4 0.05
2024-01-30
0.6 0.05
2024-02-06
3.3 0.05
2024-02-20
6.65 0.05
2024-02-27
1.45 0.05
2024-03-05
4.65 0.05
2024-03-19
No data found for midcpnifty index option 9975PE & expiry 2024-04-01.


In [80]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [32]:
tb_with_hedge = tb_with_hedge_tmp

In [33]:
tb_with_hedge["Qty"] = tb_with_hedge["Qty"] * 3
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL w cs"] + tb_with_hedge["Hedge PnL"]

In [34]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 1000000
tb_with_hedge["Entry Time"] = pd.to_datetime(tb_with_hedge["Entry Time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["Entry Time"].dt.year

In [35]:
# stats_of_trades = tb_with_hedge
# stats_of_trades["Cumulative ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Total ROI%"
# ].cumsum()
# stats_of_trades["Running Max ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Cumulative ROI%"
# ].cummax()
# stats_of_trades["Drawdown"] = (
#     stats_of_trades["Cumulative ROI%"] - stats_of_trades["Running Max ROI%"]
# )
# max_dd_distribution = (
#     stats_of_trades.groupby("Option Type")["Drawdown"].min().reset_index()
# )
# max_dd_distribution.rename(columns={"Drawdown": "Max Drawdown"}, inplace=True)
# roi_distribution = (
#     stats_of_trades.groupby("Option Type")["Total ROI%"].sum().reset_index()
# )
# grouped = pd.merge(roi_distribution, max_dd_distribution, on="Option Type")
# grouped["ROI/DD Ratio"] = grouped["Total ROI%"] / grouped["Max Drawdown"].abs()
# grouped

In [36]:
stats_df8 = pd.DataFrame(
    index=range(2023, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2023, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

/tmp/ipykernel_388085/2798960724.py:40: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2023,19.4848,13,92.3077,1.9430,-3.8309,-3.8309,5.0862
2024,37.0484,12,100.0000,3.0874,NaN,0.0000,inf
Overall,56.5331,25.0000,96.0000,2.5152,-3.8309,-3.8309,14.7571


In [86]:
a = tb_with_hedge["PnL"].sum()
b = tb_with_hedge["Hedge PnL"].sum()
c = tb_with_hedge["Total PnL"].sum()
d = tb_with_hedge["Total ROI%"].sum()
print(a, b, c, d)

523870.8340018865 -54300.78851747073 461800.1726232696 46.18001726232696


In [37]:
tb_with_hedge.to_csv('MIDCP TB RBOS w hedge 2pct_3x.csv')

In [88]:
# # tb_with_hedge['DD%'] = (tb_with_hedge['Total ROI%'].cumsum() - tb_with_hedge['Total ROI%'].cumsum().cummax())
# tb_with_hedge.drop(columns=['Cumulative ROI%', 'Running Max ROI%', 'Drawdown'], inplace=True)
# tb_with_hedge

In [89]:
# tb_with_hedge["ROI%"] = tb_with_hedge["PnL"] * 100 / 1000000

In [90]:
tb_with_hedge

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year,Hedge Strike,Hedge Contract,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Total PnL,Total ROI%
0,2023-09-05,9208.2000,8971.7500,236.4500,9241.3000,50%,9359.5250,9123.0750,low,9350,2023-09-18,6,MIDCPNIFTY239189350CE,CE,2023-09-12 09:40:00,10.8000,2023-09-18 15:30:00,0.0500,Expiry Closing,10.7500,0.1085,10.6415,2671.0757,28714.0640,28424.2523,1.4357,1.4212,2023,9725,MIDCPNIFTY239189725CE,2.0000,0.0500,-1.9500,-5208.5977,23215.6546,2.3216
1,2023-09-19,9206.3000,8975.5500,230.7500,9186.9500,50%,9302.3250,9071.5750,low,9300,2023-09-29,1,MIDCPNIFTY23SEP9300CE,CE,2023-09-28 13:14:00,3.7500,2023-09-29 15:30:00,0.0500,Expiry Closing,3.7000,0.0380,3.6620,2687.5002,9943.7506,9841.6256,0.4972,0.4921,2023,9675,MIDCPNIFTY23SEP9675CE,0.8500,0.0500,-0.8000,-2150.0001,7691.6255,0.7692
2,2023-09-26,9217.4500,8997.7500,219.7000,9112.9000,50%,9222.7500,9003.0500,low,9225,2023-10-09,5,MIDCPNIFTY2310099225CE,CE,2023-10-04 11:49:00,7.8000,2023-10-09 15:30:00,0.0500,Expiry Closing,7.7500,0.0785,7.6715,2710.6882,21007.8339,20795.0449,1.0504,1.0398,2023,9600,MIDCPNIFTY2310099600CE,1.1500,0.0500,-1.1000,-2981.7571,17813.2878,1.7813
3,2023-10-03,9130.5500,8932.3500,198.2000,9057.2500,50%,9156.3500,8958.1500,high,8950,2023-10-16,5,MIDCPNIFTY2310168950PE,PE,2023-10-11 09:15:00,13.7000,2023-10-16 15:30:00,0.0500,Friday Closing,13.6500,0.1375,13.5125,2730.3456,37269.2175,36893.7950,1.8635,1.8447,2023,8600,MIDCPNIFTY2310168600PE,2.8000,0.0500,-2.7500,-7508.4504,29385.3446,2.9385
4,2023-10-10,9222.5000,9051.7000,170.8000,9206.8500,50%,9292.2500,9121.4500,low,9300,2023-10-23,5,MIDCPNIFTY2310239300CE,CE,2023-10-18 11:12:00,7.2500,2023-10-23 15:30:00,0.1000,Expiry Closing,7.1500,0.0735,7.0765,2690.4141,19236.4605,19038.7151,0.9618,0.9519,2023,9675,MIDCPNIFTY2310239675CE,0.1000,0.0500,-0.0500,-134.5207,18904.1944,1.8904
5,2023-10-17,9213.7500,8816.4500,397.3000,8863.2500,50%,9061.9000,8664.6000,low,9050,2023-10-30,4,MIDCPNIFTY23OCT9050CE,CE,2023-10-26 09:22:00,4.2500,2023-10-30 15:30:00,0.0500,Expiry Closing,4.2000,0.0430,4.1570,2758.8033,11586.9740,11468.3455,0.5793,0.5734,2023,9400,MIDCPNIFTY23OCT9400CE,1.2500,0.0500,-1.2000,-3310.5640,8157.7815,0.8158
6,2023-10-24,8922.0500,8576.5000,345.5500,8767.2500,50%,8940.0250,8594.4750,high,8600,2023-11-06,0,MIDCPNIFTY2311068600PE,PE,2023-11-06 09:15:00,0.1500,2023-11-06 15:30:00,0.0500,Friday Closing,0.1000,0.0020,0.0980,2796.4128,279.6413,274.0485,0.0140,0.0137,2023,8250,MIDCPNIFTY2311068250PE,0.1000,0.0500,-0.0500,-139.8206,134.2278,0.0134
7,2023-10-31,8973.5500,8710.3000,263.2500,8981.8500,50%,9113.4750,8850.2250,high,8850,2023-11-13,4,MIDCPNIFTY2311138850PE,PE,2023-11-09 09:36:00,2.8000,2023-11-13 15:30:00,0.0500,Friday Closing,2.7500,0.0285,2.7215,2743.1907,7543.7745,7465.5935,0.3772,0.3733,2023,8500,MIDCPNIFTY2311138500PE,0.6000,0.0500,-0.5500,-1508.7549,5956.8386,0.5957
8,2023-11-07,9272.7000,8923.8000,348.9000,9323.5500,50%,9498.0000,9149.1000,high,9150,2023-11-20,0,MIDCPNIFTY2311209150PE,PE,2023-11-20 10:20:00,0.1000,2023-11-20 15:30:00,0.0500,Friday Closing,0.0500,0.0015,0.0485,2632.1331,131.6067,127.6585,0.0066,0.0064,2023,8775,MIDCPNIFTY2311208775PE,0.0000,0.0000,0.0000,0.0000,127.6585,0.0128
9,2023-11-21,9533.6000,9437.5500,96.0500,9525.3500,50%,9573.3750,9477.3250,high,9475,2023-12-04,5,MIDCPNIFTY2312049475PE,PE,2023-11-29 09:15:00,18.5000,2023-12-04 15:30:00,0.0500,Friday Closing,18.4500,0.1855,18.2645,2611.4092,48180.5006,47696.0842,2.4090,2.3848,2023,9100,MIDCPNIFTY2312049100PE,1.3000,0.0500,-1.2500,-3264.2616,44431.8226,4.4432


In [91]:
tb_with_hedge.to_csv('MIDCP TB RBOS w hedge 4pct_2x.csv')